In [5]:
import os

%load_ext autoreload
%autoreload 2

In [6]:
experiment_root_dir = os.path.abspath(os.path.join(os.getcwd()))
cloth_root_dir = os.path.abspath(os.path.join(os.getcwd(), "../.."))
cloth_root_dir, experiment_root_dir

('/home/ubuntu/itcoin-pcn-simulator',
 '/home/ubuntu/itcoin-pcn-simulator/experiments/2024_COMCOM')

In [9]:
from run_experiments import (
    TopologyType,
    setup_result_directories,
    setup_topology_directories,
)

seeds = [7, 13, 23, 42, 45]
capacities = [
    0.0,
    0.00001,
    0.00002,
    0.00005,
    0.0001,
    0.0002,
    0.0005,
    0.001,
    0.0013,
    0.0015,
    0.0018,
    0.002,
    0.005,
    0.01,
    0.02,
    0.05,
    0.1,
    1.0,
]

topology_type = TopologyType.SH_PCN
_, topologies_dir = setup_topology_directories(topology_type)
results_dir, results_file = setup_result_directories(2, topology_type)

topologies_dir, results_dir, results_file

(PosixPath('/home/ubuntu/itcoin-pcn-simulator/experiments/2024_COMCOM/topologies/SH_PCN'),
 PosixPath('/home/ubuntu/itcoin-pcn-simulator/experiments/2024_COMCOM/results/exp-1/SH_PCN'),
 PosixPath('/home/ubuntu/itcoin-pcn-simulator/experiments/2024_COMCOM/results/exp-1/SH_PCN/results.csv'))

In [ ]:
from experiments_runner import RebalancingMode, run_all_simulations

results = run_all_simulations(
    cloth_root_dir=cloth_root_dir,
    topologies_dir=topologies_dir,
    results_dir=results_dir,
    results_file=results_file,
    block_congestion_rates=0,
    block_sizes=4,
    capacities=[0.001, 0.002, 0.005],
    num_processess=1,
    seeds=seeds,
    simulation_ends=86400000,
    submarine_swap_thresholds=0.9,
    rebalancing=RebalancingMode.FULL,
    use_known_paths=1,
    syncs=1,
    tpss=None,
    tps_cfgs=os.path.abspath(os.path.join(experiment_root_dir, "PCN_load.txt")),
    cleanup=False,
)

Running block_congestion_rate=0, block_size=4, capacity='0.00100', num_processes=1, seed=7, simulation_end=86400000, submarine_swap_threshold=0.9, rebalancing_mode.value='Full', use_known_path=1, tps=None, tps_cfg='/home/ubuntu/itcoin-pcn-simulator/experiments/2024_COMCOM/PCN_load.txt', sync=1


[2024-07-05 14:07:27,646][INFO][analyzer.py:560] CLoTH Statistics Analyzer running at /home/ubuntu/itcoin-pcn-simulator/experiments/2024_COMCOM/results/exp-1/SH_PCN/20240705121632062-HT2VO


Running block_congestion_rate=0, block_size=4, capacity='0.00100', num_processes=1, seed=13, simulation_end=86400000, submarine_swap_threshold=0.9, rebalancing_mode.value='Full', use_known_path=1, tps=None, tps_cfg='/home/ubuntu/itcoin-pcn-simulator/experiments/2024_COMCOM/PCN_load.txt', sync=1


## BC statistics

In [13]:
from statistics_analyzer.blockchain import (
    COLS,
    _add_swap_latencies_to_blockchain_df,
    _df_from_blockchain_output,
    get_swap_latencies_from_blockchain_output,
)

BC_OUTPUT_FILE = os.path.abspath(
    os.path.join(
        cloth_root_dir,
        "experiments/2024_COMCOM/results/exp-1/SH_PCN/20240705121632062-HT2VO/blockchain_output_0.csv",
    )
)
BC_OUTPUT_FILE

'/home/ubuntu/itcoin-pcn-simulator/experiments/2024_COMCOM/results/exp-1/SH_PCN/20240705121632062-HT2VO/blockchain_output_0.csv'

In [33]:
blockchain_df = _df_from_blockchain_output(BC_OUTPUT_FILE)
blockchain_df

,confirmed,block.height,block.time,tx.type,tx.sender,tx.receiver,tx.amount,tx.start_time,tx.originator
0,1,8,476477.0,PREPARE_HTLC,30082,30083,217136,380903.61,30082
1,1,9,534802.0,CLAIM_HTLC,30082,30083,217136,477099.48,30083
2,1,12,744337.0,PREPARE_HTLC,30082,30080,449566,708032.88,30082
3,1,13,782122.0,CLAIM_HTLC,30082,30080,449566,744918.27,30080
4,1,16,981292.0,PREPARE_HTLC,30080,30083,469111,837993.76,30080
...,...,...,...,...,...,...,...,...,...
3813,1,1486,86029556.0,CLAIM_HTLC,30082,30080,402401,86013887.04,30080
3814,1,1493,86240169.0,PREPARE_HTLC,30080,30084,450465,86215490.85,30080
3815,1,1494,86303992.0,CLAIM_HTLC,30080,30084,450465,86240780.13,30084
3816,0,NaN,NaN,PREPARE_HTLC,30080,30083,407422,86340668.09,30080


In [75]:
prepare_htlcs = blockchain_df[
    blockchain_df[COLS.TX_TYPE].str.contains("PREPARE_HTLC")
].copy()
claim_htlcs = blockchain_df[
    blockchain_df[COLS.TX_TYPE].str.contains("CLAIM_HTLC")
].copy()
prepare_htlcs[COLS.SWAP_DISAMBIGUATE_IDX.value] = (
    prepare_htlcs.groupby([COLS.TX_SENDER, COLS.TX_RECEIVER, COLS.TX_AMOUNT]).cumcount()
    + 1
)
claim_htlcs[COLS.SWAP_DISAMBIGUATE_IDX.value] = (
    claim_htlcs.groupby([COLS.TX_SENDER, COLS.TX_RECEIVER, COLS.TX_AMOUNT]).cumcount()
    + 1
)

columns_of_interest = [
    COLS.TX_TYPE,
    COLS.TX_SENDER,
    COLS.TX_RECEIVER,
    COLS.TX_AMOUNT,
    COLS.BLOCK_HEIGHT,
    COLS.SWAP_DISAMBIGUATE_IDX,
]
merged = prepare_htlcs[columns_of_interest + [COLS.TX_START_TIME]].merge(
    claim_htlcs[columns_of_interest + [COLS.BLOCK_TIME]],
    how="left",
    on=[COLS.TX_SENDER, COLS.TX_RECEIVER, COLS.TX_AMOUNT, COLS.SWAP_DISAMBIGUATE_IDX],
    suffixes=["_prepare", "_claim"],
    validate="one_to_one",
)
# If the above one_to_one validation fails, something like the following may be needed: merged.groupby(['tx.sender', 'tx.receiver', 'tx.amount']).first().reset_index()
merged = merged.dropna(
    how="any", subset=[COLS.BLOCK_HEIGHT + "_claim", COLS.BLOCK_TIME]
)
merged[COLS.SWAP_LATENCY.value] = merged[COLS.BLOCK_TIME] - merged[COLS.TX_START_TIME]

merged

,tx.type_prepare,tx.sender,tx.receiver,tx.amount,block.height_prepare,swap.disambiguate.idx,tx.start_time,tx.type_claim,block.height_claim,block.time,swap.latency
0,PREPARE_HTLC,30082,30083,217136,8,1,380903.61,CLAIM_HTLC,9,534802.0,153898.39
1,PREPARE_HTLC,30082,30080,449566,12,1,708032.88,CLAIM_HTLC,13,782122.0,74089.12
2,PREPARE_HTLC,30080,30083,469111,16,1,837993.76,CLAIM_HTLC,17,996936.0,158942.24
3,PREPARE_HTLC,30082,30080,416961,27,1,1668052.78,CLAIM_HTLC,28,1694303.0,26250.22
4,PREPARE_HTLC,4275,4273,204217,29,1,1704168.58,CLAIM_HTLC,30,1742309.0,38140.42
...,...,...,...,...,...,...,...,...,...,...,...
1921,PREPARE_HTLC,30095,30093,229502,1471,1,85624882.18,CLAIM_HTLC,1472,85694772.0,69889.82
1922,PREPARE_HTLC,30084,4273,201488,1478,1,85838235.39,CLAIM_HTLC,1479,85879688.0,41452.61
1923,PREPARE_HTLC,30082,30083,200617,1483,1,85928566.41,CLAIM_HTLC,1484,85987878.0,59311.59
1924,PREPARE_HTLC,30082,30080,402401,1485,1,86004584.80,CLAIM_HTLC,1486,86029556.0,24971.20


In [80]:
blockchain_df_with_latencies = _add_swap_latencies_to_blockchain_df(blockchain_df)
blockchain_df_with_latencies

,tx.type_prepare,tx.sender,tx.receiver,tx.amount,block.height_prepare,swap.disambiguate.idx,tx.start_time,tx.type_claim,block.height_claim,block.time,swap.latency
0,PREPARE_HTLC,30082,30083,217136,8,1,380903.61,CLAIM_HTLC,9,534802.0,153898.39
1,PREPARE_HTLC,30082,30080,449566,12,1,708032.88,CLAIM_HTLC,13,782122.0,74089.12
2,PREPARE_HTLC,30080,30083,469111,16,1,837993.76,CLAIM_HTLC,17,996936.0,158942.24
3,PREPARE_HTLC,30082,30080,416961,27,1,1668052.78,CLAIM_HTLC,28,1694303.0,26250.22
4,PREPARE_HTLC,4275,4273,204217,29,1,1704168.58,CLAIM_HTLC,30,1742309.0,38140.42
...,...,...,...,...,...,...,...,...,...,...,...
1921,PREPARE_HTLC,30095,30093,229502,1471,1,85624882.18,CLAIM_HTLC,1472,85694772.0,69889.82
1922,PREPARE_HTLC,30084,4273,201488,1478,1,85838235.39,CLAIM_HTLC,1479,85879688.0,41452.61
1923,PREPARE_HTLC,30082,30083,200617,1483,1,85928566.41,CLAIM_HTLC,1484,85987878.0,59311.59
1924,PREPARE_HTLC,30082,30080,402401,1485,1,86004584.80,CLAIM_HTLC,1486,86029556.0,24971.20


In [86]:
# merged= merged.groupby(['tx.sender', 'tx.receiver', 'tx.amount'])['salary'].first().reset_index()
get_swap_latencies_from_blockchain_output(BC_OUTPUT_FILE).max()

2746139.22